In [1]:
import taichi as ti
import taichi.math as tm

import numpy as np

# running on vulkan for testing purposes, behaves the same on any optimized architecture
# ** except CPU ***
ti.init(arch=ti.vulkan)

[Taichi] version 1.7.1, llvm 15.0.4, commit 0f143b2f, linux, python 3.10.14


[I 10/25/24 09:03:43.140 8566] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


[Taichi] Starting on arch=vulkan


## Probabilties & Serialized Loops

### Creating a CDF

In [2]:
# given areas of triangles
areas = np.asarray([2, 3, 4, 2, 3, 6, 7, 10, 3, 2, 7, 2, 4, 5])


#  put them in a field
areas_field = ti.field(dtype=float, shape=(areas.shape[0]))
areas_field.from_numpy(areas)

print(areas_field)

[ 2.  3.  4.  2.  3.  6.  7. 10.  3.  2.  7.  2.  4.  5.]


In [3]:
areas_cdf = ti.field(dtype=float, shape=(areas.shape[0]))

areas_cdf_serialized = ti.field(dtype=float, shape=(areas.shape[0]))

In [4]:
@ti.kernel
def compute_cdf():
    num = 0.
    for i in ti.ndrange(areas.shape[0]):
        num += areas_field[i]
        areas_cdf[i] = num
    
    # normalize
    for i in ti.ndrange(areas.shape[0]):
        areas_cdf[i] /= num


@ti.kernel
def compute_cdf_serialized():
    num = 0.
    ti.loop_config(serialize=True)
    for i in ti.ndrange(areas.shape[0]):
        num += areas_field[i]
        areas_cdf_serialized[i] = num
    
    # normalize
    # Question: Do we need to serialize this loop?
    for i in ti.ndrange(areas.shape[0]):
        areas_cdf_serialized[i] /= num

print(areas_cdf)
compute_cdf()
print(areas_cdf)

print("========================")

print(areas_cdf_serialized)
compute_cdf_serialized()
print(areas_cdf_serialized)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.03333334 0.08333334 0.15       0.18333334 0.23333335 0.33333334
 0.45000002 0.6166667  0.6666667  0.70000005 0.8166667  0.85
 0.9166667  1.        ]


When running taichi on an optimized architecture (cuda, metal, vulkan, etc...) forloop are unpredictable. They get highly optimizied and dispatched in multiple thread, and are NOT guaranteed to run in parallel (in fact, they never will).


Using the ti.loop_config(serialized=True) will force the order of execution (but obviously slower)

### Sampling a CDF

In [5]:

@ti.kernel
def sample_cdf() -> tm.vec3:
    u = ti.random()

    return tm.vec3([sample_cdf_loop(u), sample_cdf_loop_parallel(u), sample_cdf_binary(u)])


@ti.func
def sample_cdf_loop(u:float) -> float:
    k = 0
    ti.loop_config(serialize=True)
    for i in ti.ndrange(areas.shape[0]- 1):
        if areas_cdf_serialized[i] >= u:
            k = i
            break
    return k

@ti.func
def sample_cdf_loop_parallel(u:float) -> float:
    k = 0
    # ti.loop_config(serialize=True)
    for i in ti.ndrange(areas.shape[0]- 1):
        if areas_cdf_serialized[i] < u and areas_cdf_serialized[i+1] >= u:
            k = i+1
    return k
        


@ti.func
def sample_cdf_binary(u) -> float:

    left = 0
    right = areas.shape[0]-1

    while left < right:
        mid = (left + right) // 2
        if areas_cdf_serialized[mid] < u:
            left = mid + 1
        else:
            right = mid
    return left


print(sample_cdf())

[10. 10. 10.]
